In [ ]:
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from os import listdir
from scipy.spatial import distance
from collections import OrderedDict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load pre-trained model
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output) 

# Define the feature extraction function
def extract_features(image_path, model):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    features = model.predict(image)
    return np.array(features).flatten()

# Initialize an ordered dictionary to hold image features
image_features = OrderedDict()

# Initialize list for true labels and predictions
true_labels = []
predictions = []

# Set the threshold for declaring a loop closure
loop_closure_threshold = 0.5  # This is just an example value; you'll need to tune this

# Function to process a new image
def process_new_image(image_path, model):
    # Extract the new image's features
    new_features = extract_features(image_path, model)

    # Compare the new features to recent images' features
    for timestamp, features in list(image_features.items())[-100:]:  # compare with last 100 images
        dist = distance.euclidean(new_features, features)
        if dist < loop_closure_threshold:
            print(f"Loop closure detected with image at timestamp {timestamp}")
            predictions.append(1)
            return

    # If no loop closure detected
    predictions.append(0)

    # Add the new image's features to the dictionary
    image_features[image_path] = new_features

# Load images and process them
image_directory = './images'

# You will need to supply the true labels somehow. If you have a file of true labels, you can load it like this:
# true_labels = np.load('true_labels.npy')

for image in sorted(listdir(image_directory)):
    process_new_image(image_directory + '/' + image, model)


2023-07-18 20:27:11.134016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 20:27:11.206093: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 20:27:12.751415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-18 20:27:15.307338: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-07-18 20:27:15.714640: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-07-18 20:27:15.960743: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 64373286793e3c8b2b4e3219cbf3544b so we will re-download the data.
553467096/553467096 [==============================] - 1279s 2us/step


2023-07-18 20:48:39.104581: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


1/1 [==============================] - 1s 918ms/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 978ms/step


1/1 [==============================] - 1s 894ms/step


In [3]:
# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
